In [1]:
import pandas as pd
import re

In [44]:
richmond = pd.read_excel("Richmond.xlsx",header=1, index_col=None)

In [46]:
richmond = richmond.drop([0,1,2,3,4,5]).reset_index(drop=True)

In [49]:
richmond.tail(10)

,Url,NodeId,Title,Subtitle,Content
4232,https://library.municode.com/VA/Richmond/codes...,CH30ZO_ARTXIAM_DIV2COZO_S30-1170.9AMVACO,Sec. 30-1170.9.,Amendments and variations of conditions.,Amendments and variations of conditions attach...
4233,https://library.municode.com/VA/Richmond/codes...,CH30ZO_ARTXIIDE,ARTICLE XII.,DEFINITIONS,NaN
4234,https://library.municode.com/VA/Richmond/codes...,CH30ZO_ARTXIIDE_S30-1200APAR,Sec. 30-1200.,Applicability of article.,"For the purposes of this chapter, certain word..."
4235,https://library.municode.com/VA/Richmond/codes...,CH30ZO_ARTXIIDE_S30-1210GERUIN,Sec. 30-1210.,General rules of interpretation.,(a)\n\n For the purposes of this...
4236,https://library.municode.com/VA/Richmond/codes...,CH30ZO_ARTXIIDE_S30-1220DE,Sec. 30-1220.,Definitions.,Content is too large for cell.
4237,https://library.municode.com/VA/Richmond/codes...,CD_COMPARATIVE_TABLE1993CO,CODE COMPARATIVE TABLE,1993 CODE,Content is too large for cell.
4238,https://library.municode.com/VA/Richmond/codes...,CD_COMPARATIVE_TABLE2004CO,CODE COMPARATIVE TABLE,2004 CODE,Content is too large for cell.
4239,https://library.municode.com/VA/Richmond/codes...,CD_COMPARATIVE_TABLE2015CO,CODE COMPARATIVE TABLE,2015 CODE,Content is too large for cell.
4240,https://library.municode.com/VA/Richmond/codes...,CD_COMPARATIVE_TABLELE,CODE COMPARATIVE TABLE,LEGISLATION,Content is too large for cell.
4241,https://library.municode.com/VA/Richmond/codes...,STLARETA,STATE LAW REFERENCE TABLE,NaN,Content is too large for cell.


In [50]:
richmond['Url'][4240]

'https://library.municode.com/VA/Richmond/codes/Code_of_Ordinances?nodeId=CD_COMPARATIVE_TABLELE'

# Counties

In [51]:
counties = pd.read_csv('counties.csv')

In [52]:
counties

,Accomack
0,Albemarle
1,Alexandria
2,Alleghany
3,Amelia
4,Amherst
...,...
127,Williamsburg
128,Winchester
129,Wise
130,Wythe


# BeautifulSoup4 (doesn't work)

In [70]:
import requests 
url = "https://library.municode.com/VA/Richmond/codes/Code_of_Ordinances?nodeId=STLARETA"
headers = {'user-agent': 'Kropko class example (hab6xf@virginia.edu)'}
r = requests.get(url, headers=headers)
r

<Response [200]>

In [149]:
from bs4 import BeautifulSoup

richmondCodes = BeautifulSoup(r.text, 'html.parser')

In [150]:
tdlist = richmondCodes.find_all("tbody", " top ")
tdlist

[]

In [151]:
import requests 
url = "https://library.municode.com/VA/Richmond/codes/Code_of_Ordinances?nodeId=STLARETA"
headers = {'user-agent': '(hab6xf@virginia.edu)'}
r = requests.get(url, headers=headers)

from bs4 import BeautifulSoup

richmondCodes = BeautifulSoup(r.text, 'html.parser')

richmondCodes


<!DOCTYPE html>

<html lang="en" ng-app="mcc.library_desktop" ng-strict-di="true">
<head>
<title>Municode Library</title>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport">
<meta content="municipal, codes, codification" name="keywords"/>
<meta content="MunicodeNEXT, the industry's leading search application with over 3,300 codes and growing!" name="description"/>
<base href="https://library.municode.com/"/>
<link href="dist/img/apple-touch-icon-144x144-precomposed.png" rel="apple-touch-icon-precomposed" sizes="144x144"/>
<link href="dist/img/apple-touch-icon-114x114-precomposed.png" rel="apple-touch-icon-precomposed" sizes="114x114"/>
<link href="dist/img/apple-touch-icon-72x72-precomposed.png" rel="apple-touch-icon-precomposed" sizes="72x72"/>
<link href="dist/img/apple-touch-icon-57x57-precomposed.png" rel="apple-touch-icon-precomposed"/>
<link href="dist/img/apple-touch-icon.

# Selenium (works)
- using selenium webdriver, xpath, outerHTML, and pandas dataframe

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [238]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

driver = webdriver.Chrome('/Users/holdenbruce/Downloads/chromedriver')

# set implicit wait time so that apis/javascript load before we scrape 
driver.implicitly_wait(10) # seconds

town = "Richmond"

# url of the county
url = f"https://library.municode.com/VA/{town}/codes/Code_of_Ordinances?nodeId=STLARETA"
headers = {'user-agent': 'class project (hab6xf@virginia.edu)'}


# xpath of the table in the webpage created by javascript 
xpath = "/html/body/div[1]/div[2]/ui-view/mcc-codes/div[2]/section/div[1]/mcc-codes-content/div/div[2]/ul/li/mcc-codes-content-chunk/div/div/div[2]/div/div/div/div[2]/table"
driver.get(url)

# use xpath to get to the table
data = driver.find_elements_by_xpath(xpath)


In [239]:
type(data)

list

In [240]:
# use outerHTML to maintain the html/css/javasript code pulled from the webpage 
table = data[0].get_attribute("outerHTML")

# https://stackoverflow.com/questions/41214702/parse-html-and-read-html-table-with-selenium-python

In [409]:

# convert that table into a pandas dataframe
df = pd.read_html(table)
df = df[0]

In [410]:
df

,Code of Virginia Section,Section this Code
0,1-210,1-2
1,1-216,1-2
2,1-217,1-8
3,1-222,1-2
4,1-223,1-2
...,...,...
1182,—,22-239—22-241
1183,—,22-299
1184,—,22-317
1185,—,26-956


In [243]:
df.columns

Index(['Code of Virginia  Section', 'Section this Code'], dtype='object')

In [251]:
# rename the columns
df = df.rename(columns={'Code of Virginia  Section': "Virginia", "Section this Code":town})
df

,Virginia,Richmond
0,1-210,1-2
1,1-216,1-2
2,1-217,1-8
3,1-222,1-2
4,1-223,1-2
...,...,...
1182,—,22-239—22-241
1183,—,22-299
1184,—,22-317
1185,—,26-956


In [252]:
# now write to CSV

import os  
df.to_csv(f'countyCSV/{town}.csv', index=False)  

### find list of town urls

In [3]:
def list_of_towns():
    driver = webdriver.Chrome('/Users/holdenbruce/Downloads/chromedriver')

    # set implicit wait time so that apis/javascript load before we scrape 
    driver.implicitly_wait(10) # seconds

    # url of the county
    url = f"https://library.municode.com/VA"
    # headers to let them know who i am
    headers = {'user-agent': 'class project (hab6xf@virginia.edu)'}
    # xpath of the table in the webpage created by javascript 
    xpathHOME = "/html/body/div[1]/div[2]/ui-view/div[2]/section/div/div"

    driver.get(url)

    # use xpath to get to the table
    data = driver.find_elements_by_xpath(xpathHOME)
    # links = driver.find_elements_by_tag_name("a")
    
    # add a delay of 3 seconds in the function
    time.sleep(3)
    
    # use outerHTML to maintain the html/css/javasript code pulled from the webpage 
    html = data[0].get_attribute("outerHTML")

    r1 = re.findall(r"https://library.municode.com/VA/[\w\.-]+",html)

    # convert to dataframe and drop duplicates
    towns_list = pd.DataFrame(r1)
    towns_list = towns_list.drop_duplicates().reset_index(drop=True)

    return towns_list

In [423]:
list_of_towns = list_of_towns()
list_of_towns

,0
0,https://library.municode.com/VA/accomack_county
1,https://library.municode.com/VA/albemarle_county
2,https://library.municode.com/VA/alexandria
3,https://library.municode.com/VA/alleghany_county
4,https://library.municode.com/VA/altavista
...,...
141,https://library.municode.com/VA/winchester
142,https://library.municode.com/VA/windsor
143,https://library.municode.com/VA/wise_county
144,https://library.municode.com/VA/woodstock


# Automate 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

def scraper(town):
    driver = webdriver.Chrome('/Users/holdenbruce/Downloads/chromedriver')

    # set implicit wait time so that apis/javascript load before we scrape 
    driver.implicitly_wait(10) # seconds

    # url of the county
    url = f"https://library.municode.com/VA/{town}/codes/Code_of_Ordinances?nodeId=STLARETA"
    headers = {'user-agent': 'class project (hab6xf@virginia.edu)'}


    # xpath of the table in the webpage created by javascript 
    xpath = "/html/body/div[1]/div[2]/ui-view/mcc-codes/div[2]/section/div[1]/mcc-codes-content/div/div[2]/ul/li/mcc-codes-content-chunk/div/div/div[2]/div/div/div/div[2]/table"
    driver.get(url)

    # use xpath to get to the table
    data = driver.find_elements_by_xpath(xpath)


In [253]:
# read in counties
counties = pd.read_csv('counties.csv')

# loop through each county and call the scraper 
counties

,Accomack
0,Albemarle
1,Alexandria
2,Alleghany
3,Amelia
4,Amherst
...,...
127,Williamsburg
128,Winchester
129,Wise
130,Wythe


# Identify URL for State Comparison Table

In [377]:
def identify_comparison_table_URL_part1(town):
    driver = webdriver.Chrome('/Users/holdenbruce/Downloads/chromedriver')

    # set implicit wait time so that apis/javascript load before we scrape 
    driver.implicitly_wait(10) # seconds

    # url of the county
    url = f"https://library.municode.com/va/{town}/codes/code_of_ordinances"
    # headers to let them know who i am
    headers = {'user-agent': 'class project (hab6xf@virginia.edu)'}
    # xpath of the table in the webpage created by javascript 
    xpath = "/html/body/div[1]/div[2]/ui-view/mcc-codes/div[2]/section[1]/div[2]"

    driver.get(url)

    # use xpath to get to the table
    data = driver.find_elements_by_xpath(xpath)
    # links = driver.find_elements_by_tag_name("a")
    
    return data

In [381]:
data = identify_comparison_table_URL_part1(town)

In [391]:
def outerHTML_part2(data):
    # use outerHTML to maintain the html/css/javasript code pulled from the webpage 
    html = data[0].get_attribute("outerHTML")

    r2 = re.findall(r'(https?://[^\s]+)', html)
    
    url = r2[-1]
    url = url[:-1]
    
    return url
    # "https://library.municode.com/va/Richmond/codes/code_of_ordinances?nodeId=STLARETA" 

In [392]:
url = outerHTML_part2(data)
url

'https://library.municode.com/va/Richmond/codes/code_of_ordinances?nodeId=STLARETA'

In [401]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

def scraper(url):
    driver = webdriver.Chrome('/Users/holdenbruce/Downloads/chromedriver')

    # set implicit wait time so that apis/javascript load before we scrape 
    driver.implicitly_wait(10) # seconds

    # url of the county

    # xpath of the table in the webpage created by javascript 
    xpath = "/html/body/div[1]/div[2]/ui-view/mcc-codes/div[2]/section/div[1]/mcc-codes-content/div/div[2]/ul/li/mcc-codes-content-chunk/div/div/div[2]/div/div/div/div[2]/table"
    driver.get(url)

    # use xpath to get to the table
    data = driver.find_elements_by_xpath(xpath)
    
    # use outerHTML to maintain the html/css/javasript code pulled from the webpage 
    table = data[0].get_attribute("outerHTML")

    # https://stackoverflow.com/questions/41214702/parse-html-and-read-html-table-with-selenium-python

    # convert that table into a pandas dataframe
    df = pd.read_html(table)
    df = df[0]


    # rename the columns
    df = df.rename(columns={'Code of Virginia  Section': "Virginia", "Section this Code":town})


    # now write to CSV
    df.to_csv(f'countyCSV/{town}.csv', index=False)  
    
    return df

In [402]:
# scrapedData = 
scraper(url)

,Virginia,Richmond
0,1-210,1-2
1,1-216,1-2
2,1-217,1-8
3,1-222,1-2
4,1-223,1-2
...,...,...
1182,—,22-239—22-241
1183,—,22-299
1184,—,22-317
1185,—,26-956


# Combine into 1 function

In [2]:
def list_of_towns():
    driver = webdriver.Chrome('/Users/holdenbruce/Downloads/chromedriver')

    # set implicit wait time so that apis/javascript load before we scrape 
    driver.implicitly_wait(10) # seconds

    # url of the county
    url = f"https://library.municode.com/VA"
    # headers to let them know who i am
    headers = {'user-agent': 'class project (hab6xf@virginia.edu)'}
    # xpath of the table in the webpage created by javascript 
    xpathHOME = "/html/body/div[1]/div[2]/ui-view/div[2]/section/div/div"

    driver.get(url)

    # use xpath to get to the table
    data = driver.find_elements_by_xpath(xpathHOME)
    # links = driver.find_elements_by_tag_name("a")
    
    # add a delay of 3 seconds in the function
    time.sleep(3)
    
    # use outerHTML to maintain the html/css/javasript code pulled from the webpage 
    html = data[0].get_attribute("outerHTML")

    r1 = re.findall(r"https://library.municode.com/VA/[\w\.-]+",html)

    # convert to dataframe and drop duplicates
    towns_list = pd.DataFrame(r1)
    towns_list = towns_list.drop_duplicates().reset_index(drop=True)

    return towns_list

In [23]:
def identify_comparison_table_URL_part1(town):
    driver = webdriver.Chrome('/Users/holdenbruce/Downloads/chromedriver')

    # set implicit wait time so that apis/javascript load before we scrape 
    driver.implicitly_wait(10) # seconds

    # url of the county
    url = f"https://library.municode.com/va/{town}/codes/code_of_ordinances"
    # headers to let them know who i am
    headers = {'user-agent': 'class project (hab6xf@virginia.edu)'}
    # xpath of the table in the webpage created by javascript 
    xpath = "/html/body/div[1]/div[2]/ui-view/mcc-codes/div[2]/section[1]/div[2]"

    driver.get(url)

    # use xpath to get to the table
    data = driver.find_elements_by_xpath(xpath)
    # links = driver.find_elements_by_tag_name("a")
    
    
    # add a delay of 3 seconds in the function
    time.sleep(3)
    
    
    # use outerHTML to maintain the html/css/javasript code pulled from the webpage 
    html = data[0].get_attribute("outerHTML")

    r2 = re.findall(r'(https?://[^\s]+)', html)
    
    url = r2[-1]
    url = url[:-1]
    
    return url
    
    

In [405]:
identify_comparison_table_URL_part1(town)

'https://library.municode.com/va/Richmond/codes/code_of_ordinances?nodeId=STLARETA'

In [22]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

def scraper(town):
    driver = webdriver.Chrome('/Users/holdenbruce/Downloads/chromedriver')

    # set implicit wait time so that apis/javascript load before we scrape 
    driver.implicitly_wait(10) # seconds

    # url of the county
    url = identify_comparison_table_URL_part1(town)

    # xpath of the table in the webpage created by javascript 
    xpath = "/html/body/div[1]/div[2]/ui-view/mcc-codes/div[2]/section/div[1]/mcc-codes-content/div/div[2]/ul/li/mcc-codes-content-chunk/div/div/div[2]/div/div/div/div[2]/table"
    driver.get(url)

    # use xpath to get to the table
    data = driver.find_elements_by_xpath(xpath)
    
    # use outerHTML to maintain the html/css/javasript code pulled from the webpage 
    table = data[0].get_attribute("outerHTML")

    # https://stackoverflow.com/questions/41214702/parse-html-and-read-html-table-with-selenium-python

    # convert that table into a pandas dataframe
    df = pd.read_html(table)
    df = df[0]


    # rename the columns
    df = df.rename(columns={'Code of Virginia  Section': "Virginia", "Section this Code":town})


    # now write to CSV
    df.to_csv(f'countyCSV/{town}.csv', index=False)  
    
    return df

In [407]:
scraper(town)

,Virginia,Richmond
0,1-210,1-2
1,1-216,1-2
2,1-217,1-8
3,1-222,1-2
4,1-223,1-2
...,...,...
1182,—,22-239—22-241
1183,—,22-299
1184,—,22-317
1185,—,26-956


# FINAL run for all

In [1]:
import pandas as pd 
import os 
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import requests


In [2]:
def list_of_towns():
    driver = webdriver.Chrome('/Users/holdenbruce/Downloads/chromedriver')

    # set implicit wait time so that apis/javascript load before we scrape 
    driver.implicitly_wait(5) # seconds

    # url of the county
    url = f"https://library.municode.com/VA"
    # headers to let them know who i am
    headers = {'user-agent': 'class project (hab6xf@virginia.edu)'}
    # xpath of the table in the webpage created by javascript 
    xpathHOME = "/html/body/div[1]/div[2]/ui-view/div[2]/section/div/div"

    driver.get(url)

    # use xpath to get to the table
    data = driver.find_elements_by_xpath(xpathHOME)
    # links = driver.find_elements_by_tag_name("a")
    
    # add a delay of 3 seconds in the function
    time.sleep(2)
    
    # use outerHTML to maintain the html/css/javasript code pulled from the webpage 
    html = data[0].get_attribute("outerHTML")

    r1 = re.findall(r"https://library.municode.com/VA/[\w\.-]+",html)

    # convert to dataframe and drop duplicates
    towns_list = pd.DataFrame(r1)
    towns_list = towns_list.drop_duplicates().reset_index(drop=True)

    return towns_list

In [3]:
town_urls = list_of_towns()
town_urls = town_urls.rename(columns={0:"urls"})
town_urls

,urls
0,https://library.municode.com/VA/accomack_county
1,https://library.municode.com/VA/albemarle_county
2,https://library.municode.com/VA/alexandria
3,https://library.municode.com/VA/alleghany_county
4,https://library.municode.com/VA/altavista
...,...
141,https://library.municode.com/VA/winchester
142,https://library.municode.com/VA/windsor
143,https://library.municode.com/VA/wise_county
144,https://library.municode.com/VA/woodstock


In [4]:
def identify_comparison_table_URL_part1(url):
    driver = webdriver.Chrome('/Users/holdenbruce/Downloads/chromedriver')

    # set implicit wait time so that apis/javascript load before we scrape 
    driver.implicitly_wait(5) # seconds

    # url of the county
    # url = f"https://library.municode.com/va/{town}/codes/code_of_ordinances"
    # headers to let them know who i am
    # headers = {'user-agent': 'class project (hab6xf@virginia.edu)'}
    # xpath of the table in the webpage created by javascript 
    xpath = "/html/body/div[1]/div[2]/ui-view/mcc-codes/div[2]/section[1]/div[2]"

    driver.get(url)

    # use xpath to get to the table
    data = driver.find_elements_by_xpath(xpath)
    # links = driver.find_elements_by_tag_name("a")
    
    
    # add a delay of 3 seconds in the function
    time.sleep(2)
    
    
    # use outerHTML to maintain the html/css/javasript code pulled from the webpage 
    html = data[0].get_attribute("outerHTML")

    r2 = re.findall(r'(https?://[^\s]+)', html)
    
    url = r2[-1]
    url = url[:-1]
    
    return url

In [5]:
# identify_comparison_table_URL_part1(town_urls.loc[0,'urls'])

In [6]:
def scraper(url,town):
    driver = webdriver.Chrome('/Users/holdenbruce/Downloads/chromedriver')

    # set implicit wait time so that apis/javascript load before we scrape 
    driver.implicitly_wait(5) # seconds

    # url of the county
    url = identify_comparison_table_URL_part1(url)

    # xpath of the table in the webpage created by javascript 
    xpath = "/html/body/div[1]/div[2]/ui-view/mcc-codes/div[2]/section/div[1]/mcc-codes-content/div/div[2]/ul/li/mcc-codes-content-chunk/div/div/div[2]/div/div/div/div[2]/table"
    driver.get(url)

    # use xpath to get to the table
    data = driver.find_elements_by_xpath(xpath)
    
    # use outerHTML to maintain the html/css/javasript code pulled from the webpage 
    table = data[0].get_attribute("outerHTML")

    # https://stackoverflow.com/questions/41214702/parse-html-and-read-html-table-with-selenium-python

    # convert that table into a pandas dataframe
    df = pd.read_html(table)
    df = df[0]


    # rename the columns
    df = df.rename(columns={'Code of Virginia  Section': "Virginia", "Section this Code":town})


    # now write to CSV
    df.to_csv(f'countyCSV/{town}.csv', index=False)  
    
    return df

In [7]:
# scraper('https://library.municode.com/va/accomack_county/codes/code_of_ordinances?nodeId=STLARETA',"albemarle_county")

In [13]:
# loop through all of the towns 
# for ind in list_of_towns.index:
for ind in town_urls.index:
    url = town_urls.loc[ind,'urls']
    town = town_urls.loc[ind,'urls'].split("/")[-1]
    
    try:
        print(scraper(url,town))
    except:
        print(f'This municipality {town} does not host any other documents online.')
        

                             0                  1
0    Code of Virginia  Section  Section this Code
1                 1-13 et seq.              Ch. 1
2                       1-13.3                1-2
3               1-13.6, 1-13.7                1-2
4                       1-13.9                1-4
..                         ...                ...
401              62.1-44.15:46              38-98
402           62.1-148 et seq.   Ch. 102, Art. II
403                   62.1-255            106-376
404                   63.1-164              22-88
405          68.1-1200 et seq.              22-88

[406 rows x 2 columns]
This municipality albemarle_county does not host any other documents online.
This municipality alexandria does not host any other documents online.
                              0                  1
0    Code of  Virginia  Section  Section this Code
1                  1-13 et seq.                1-2
2                        1-13.9                1-3
3              2.2-3700 

list_of_townsIn situations like https://library.municode.com/VA/albemarle_county added functionality could be to look at dimensions of dataframe and have that affect how data is stored and used. If it's a 2x_ matrix then carry on as is but if it's a 4x_ matrix then only take the first and last column (as in the albemarle county example

amherst_county has another weird thing happen: there are multiple tables in that webpage and i took the wrong one ... how can we make sure that we are taking the talbe with the most useful/relevant information to our usecase? 